# Find COVID-19 Mutations

The purpose of this notebook is find mutations in COVID-19 strains. This analyze depends on strain data compiled by the `process_strains.ipynb` notebook and stored in the `pickles` folder. 

Notebook workflow:
1. Open pickle folder and get sequences of the oringal strain to be compared to 
2. Open pickle folder and get sequences for all other strains
3. Comapre the amino acid sequence of each strain to the orignal and save found differences as mutations
4. Re-save the picle files along with an new field for each feature called `mutants` for mutations

### Imports

Begin workflow by importing the open source Python libraries to be used.

In [19]:
try:
  from Bio import pairwise2
except ImportError as e:
  !pip install biopython
  from Bio import pairwise2
import pickle
import os
import pprint
pp = pprint.PrettyPrinter(indent=4)
from Bio.pairwise2 import MAX_ALIGNMENTS

print('Library imports complete')

Library imports complete


### Configs

Set the config variables as needed.

In [20]:
original_strain = 'NC_045512.2'

pickles_path = '../pickles/'

amino_acid_colors = {
  'T': 'green',
  'Q': 'green',
  'S': 'green',
  'N': 'green',
  'F': 'blue',
  'M': 'blue',
  'L': 'blue',
  'V': 'blue',
  'W': 'blue',
  'A': 'blue',
  'I': 'blue',
  'E': 'purple',
  'D': 'purple',
  'K': 'red',
  'R': 'red',
  'C': 'peach',
  'G': 'peach',
  'P': 'yellow',
  'H': 'teal',
  'Y': 'teal'
 }  
amino_acid_types = {
  'T': 'polar',
  'Q': 'polar',
  'S': 'polar',
  'N': 'polar',
  'F': 'nonpolar',
  'M': 'nonpolar',
  'L': 'nonpolar',
  'V': 'nonpolar',
  'W': 'nonpolar',
  'A': 'nonpolar',
  'I': 'nonpolar',
  'E': 'negative',
  'D': 'negative',
  'K': 'positive',
  'R': 'positive',
  'C': 'nonpolar',
  'G': 'nonpolar',
  'P': 'nonpolar',
  'H': 'positive',
  'Y': 'polar'
 }

print('Config variables set')

Config variables set


### Original strain features

Here we retrieve the found features that were stored previously in pickle files by the `process_strains.ipynb` notebook. In order to compare a feature accross strains, we need to make use of the feature `product` name. Each feature from NCBI has product name and nucleotide start-end coordinates. We can restructure the feature info into a dictionary using product name for lookup keys.

In [21]:
# Open original strain pickle file of feature data generated by
strain = original_strain
pickles_file = strain + '.pickle'
filepath = pickles_path + pickles_file
objects = []
with (open(filepath, "rb")) as openfile:
  while True:
    try:
      objects.append(pickle.load(openfile))
    except EOFError:
      break
orig_feats = objects[0]

# Scan the info about each feature and store it to a dictionary named `original` indexed by its `product` name
original = {}
for feat in orig_feats:
  product = feat['product']
  if product not in original:
    original[product] = feat

# Print the names of features found
print('Original strain {} features read:'.format(original_strain))
for product in original.keys():
  print('  ', product)

Original strain NC_045512.2 features read:
   leader protein
   nsp2
   nsp3
   nsp4
   3C-like proteinase
   nsp6
   nsp7
   nsp8
   nsp9
   nsp10
   helicase
   3'-to-5' exonuclease
   endoRNAse
   2'-O-ribose methyltransferase
   ORF1a polyprotein
   nsp11
   surface glycoprotein
   ORF3a protein
   envelope protein
   membrane glycoprotein
   ORF6 protein
   ORF7a protein
   ORF7b
   ORF8 protein
   nucleocapsid phosphoprotein
   ORF10 protein


### Features for the other strains

Now we want to do the same for other strains. These are kept in their own list. The original is separate because the data each of the strains in this new list will be compared to the same original strain. Deviations from the amino acid sequence of the original strain will be considered mutations.

In [22]:
for file_name in os.listdir(pickles_path):
  if '.pickle' not in file_name:
    continue
  comparison_strain = file_name.replace('.pickle', '')
  if comparison_strain == original_strain:
    continue
  strain = comparison_strain
  pickles_file = strain + '.pickle'
  filepath = pickles_path + pickles_file
  objects = []
  with (open(filepath, "rb")) as openfile:
    while True:
      try:
        objects.append(pickle.load(openfile))
      except EOFError:
        break
  comp_feats = objects[0]

  comparison = {}
  for feat in comp_feats:
    product = feat['product']
    # rewrite in case of repeats should be same
    if product not in comparison:
      comparison[product] = feat

  mutations_count = 0
  for key, comp_feat in comparison.items():
    orig_feat = original[key]
    if 'translation' not in comp_feat:
      continue
    alignments = pairwise2.align.globalxx(orig_feat['translation'], comp_feat['translation'])
    max_alignment = None
    max_score = 0
    for alignment in alignments:
      if alignment.score > max_score:
        max_score = alignment.score
        max_alignment = alignment
  
    str1 = max_alignment.seqA
    str2 = max_alignment.seqB
    positions = [] 
    for x in range (0, min(len(str1), len(str2))): 
      if str1[x].upper() != str2[x].upper(): 
        positions.append(x)

    diffs = []
    for position in positions:
      coord = position + 1
      diffs.append( (coord, str1[position], str2[position]) )
    comp_feat['diffs'] = diffs

  for comp_feat in comparison.values():
    diffs = []
    if 'diffs' in comp_feat:
      diffs = comp_feat['diffs']
    else:
      comp_feat['diffs'] = diffs
    mutants = []
    for idx in range(len(diffs) - 1):
      left = diffs[idx][0]
      right = diffs[idx + 1][0]
      if diffs[idx][1] != '-':
        orig_amino_acid = diffs[idx][1]
      else:
        orig_amino_acid = diffs[idx][2]
      if diffs[idx + 1][2] != '-':
        comp_amino_acid = diffs[idx + 1][2]
      else:
        comp_amino_acid = diffs[idx + 1][1]
      if '*' in [orig_amino_acid, comp_amino_acid]:
        continue
      if right - left == 1:
        colors = amino_acid_colors
        mutant_color_change = 'same'
        if 'X' in [orig_amino_acid, comp_amino_acid]:
          mutant_color_change = 'X may differ'
        elif colors[orig_amino_acid] != colors[comp_amino_acid]:
          mutant_color_change = '{} to {}'.format(colors[orig_amino_acid], colors[comp_amino_acid])
        types = amino_acid_types
        mutant_change_type = 'conservative'
        if 'X' in [orig_amino_acid, comp_amino_acid]:
          mutant_change_type = 'X may not be conserved'
        elif types[orig_amino_acid] != types[comp_amino_acid]:
          mutant_change_type = '{} to {}'.format(types[orig_amino_acid], types[comp_amino_acid])
        mutants.append( (left, orig_amino_acid, comp_amino_acid, mutant_color_change, mutant_change_type) )
        mutations_count += 1
    comp_feat['mutants'] = mutants

  # print the features found for strain
  print('Mutations found for {} strain: {}'.format(strain, mutations_count))

  comparisons = []
  for comp in comparison.values():
    comparisons.append(comp)
    
  with open(filepath, 'wb') as file:
    pickle.dump(comparisons, file)

print('\nFinding mutations and re-saving files complete')

Mutations found for MW243586.1 strain: 23
Mutations found for OP733821.1 strain: 57
Mutations found for OK341237.1 strain: 7
Mutations found for OM251163.1 strain: 73
Mutations found for OQ050563.1 strain: 6
Mutations found for MW474188.1 strain: 1
Mutations found for OL947440.1 strain: 131
Mutations found for OQ253610.1 strain: 66

Finding mutations and re-saving files complete


### View mutation counts for a strain

Here you can select a single strain you want to see found mutations for.

In [23]:
# change this to strain you want to see counts for
strain = 'MW474188.1'

pickles_file = strain + '.pickle'
filepath = pickles_path + pickles_file
objects = []
with (open(filepath, "rb")) as openfile:
  while True:
    try:
      objects.append(pickle.load(openfile))
    except EOFError:
      break
comparison_feats = objects[0]

print('Amino acid mutation counts found in {} from the original {} strain:'.format(strain, original_strain))
for feat in sorted(comparison_feats, key=lambda d: len(d['mutants']), reverse=True):
  print('  {}: {}'.format(feat['product'], len(feat['mutants'])))

Amino acid mutation counts found in MW474188.1 from the original NC_045512.2 strain:
  surface glycoprotein: 1
  leader protein: 0
  nsp2: 0
  nsp3: 0
  nsp4: 0
  3C-like proteinase: 0
  nsp6: 0
  nsp7: 0
  nsp8: 0
  nsp9: 0
  nsp10: 0
  helicase: 0
  3'-to-5' exonuclease: 0
  endoRNAse: 0
  2'-O-ribose methyltransferase: 0
  ORF1a polyprotein: 0
  nsp11: 0
  ORF3a protein: 0
  envelope protein: 0
  membrane glycoprotein: 0
  ORF6 protein: 0
  ORF7a protein: 0
  ORF7b: 0
  ORF8 protein: 0
  nucleocapsid phosphoprotein: 0
  ORF10 protein: 0
